[Accueil](../../../index.ipynb) > [Sommaire](../../index.ipynb) > [6.4 Programmation dynamique](index.ipynb)

# TP Rendu de monnaie

L'optimisation du **rendu de monnaie** a déjà été traitée en première grâce [aux algorithmes gloutons](../../../Premiere/8_Algorithmique/5_algorithme_glouton.ipynb).

**Nous allons voir que la résolution par des algorithmes gloutons de donne pas toujours la solution optimale et parfois ne trouve pas la solution...**

- Vous avez à votre disposition différentes pièces de monnaie;
- Vous devez rendre une certaine somme en minimisant le nombre de pièces rendues.

La solution par l'algorithme glouton est la suivante:

- Prendre la plus grande pièce possible;
- Recommencer l'opération jusqu'à ce que la somme à rendre soit nulle.

**Exemple 1**:

Vous avez à votre disposition les pièces suivantes:

- 2 centimes
- 5 centimes
- 10 centimes
- 50 centimes.

Vous devez rendre **11** centimes.

On applique l'algorithme glouton:

- On utilise une pièce de 10 centimes;
- Il reste 1 centime à rendre, mais nous ne disposons pas de cette pièce...

Pourtant la solution existe : 1 pièce de 5 et 3 pièces de 2.

**Exemple 2**:

Dans l’ancien système monétaire britannique qui avait cours avant 1971, **la livre sterling était divisée en 20 shillings** divisés chacun en 12 pence (pluriel de penny).

La valeur des différentes pièces existantes était 1, 3, 4, 6, 12, 24, 30, 60, 120, 240 et 252 pence. Depuis 1971, c’est un système décimal qui est utilisé avec les mêmes valeurs que dans le système de l’euro.

Si on applique l'algorithme glouton pour rendre 49 pence on trouve:

- 1 pièce de 30 pence (reste 19)
- 1 pièce de 12 pence (reste 7)
- 1 pièce de 6 pence (reste 1)
- 1 pièce de 1 penny (reste 0)

On a donc utilisé 4 pièces.

Pourtant la solution optimale est 2 pièces de 24 et 1 pièce de 1, soit 3 pièces.

<div class="alert alert-info">Dans certains cas un algorithme glouton trouvera une solution, mais celle ci ne sera pas forcément optimale.</div>

## Modélisation du problème.

On a: 

- $x$ la **somme à rendre**;
- $N(x)$ **le nombre de pièces** à rendre.

Si je suis capable de rendre $x$ centimes avec $N(x)$ pièces, quelle somme suis-je capable de rendre avec $N(x)+1$ pièces ?

Je dispose des pièces suivantes ${p_1, p_2, p_3,...., p_n}$. 

Si je suis capable de rendre $x$ centimes alors je suis capable de rendre:

- $x-p_1$ si $x \geq p_1$
- $x-p_2$ si $x \geq p_2$
- ...
- $x-p_1$ si $x \geq p_i$
- ...
- $x-p_n$ si $x \geq p_n$

Autrement dit $N(x) = 1 + N(x-p_i)\;\forall\;x\;\geq\;p_i$

**Exemple** : Il me faudra au moins une pièce de moins pour rendre 15 centimes que 20 centimes, si dispose de pièces inférieures ou égales à 5 centimes.

Ce qui nous intéresse c'est le **nombre minimum de pièces** parmi toutes les valeurs de $N(x-p_i)$.

La relation de récurrence pour notre problème d'optimisation est donc:

$
N(x)=
\left\{
    \begin{array}{ll}
        0\;si\; x=0 \\
        1+min(N(x-p_i))\; avec\;1\leq i <n\;et\;p_i\leq\;x
    \end{array}
\right.
$

### Première solution : la force brute.

<div class="alert alert-info">

**A faire** : Implémenter cette solution en Python

</div>

pseudo code:
```
fonction rendu_monnaie(pieces, rendu)
    nb_min <- infini
    #cas d'arrêt
    si le rendu est nul, on retourne 0
    #cas général
    pour chaque pièce
        si la pièce est inférieure au rendu
            nb_pieces <- 1 + rendu_monnaie(pieces, rendu - piece)
            si nb_pieces est inférieur à nb_min:
                nb_min <- nb_pieces
            fin de si
        fin de si
    fin pour
    on retourne nb_min

```

In [ ]:
P = [2, 5, 10, 50, 100]

def rendu_monnaie(P, rendu):
    """
    P : la liste des pièces disponibles
    rendu : le montant à rendre
    Retourne le nombre minimal de pièces à rendre
    """
    # On initialise le minimum à l'infini
    nb_min = float('inf')
    pass
    # On retourne le nombre mini trouvé
    return nb_min
        
    
rendu_monnaie(P, 9)

### Observation de l'arbre d'appel récursif

- Construire l'arbre des appels récursif pour un rendu de 9.
- Existe t-il des redondances dans l'arbre ?
- Essayez pour le rendu de 90 centimes.
- Quels sont les avantages et inconvénients de cet algorithme par **force brute** ?

### Utilisation de la programmation dynamique : mémoïsation

Nous allons, cette fois-ci, utiliser la mémoïsation afin de ne pas recalculer des solutions déjà calculées.

Une fois la fonction écrite, calculer le nombre de pièces optimal pour rendre 1,71€.


**A faire**:

Essayer le rendu pour 49 pence.

In [ ]:
# Voici le systeme des pences
P=[1, 3, 4, 6, 12, 24, 30, 60, 120, 240, 252]
